# NFL Team Data Collection and Data Cleaning

In this file the code to collect NFL team statistics and team records to perform analysis is collected. The data is then cleaned to perform further analysis of this data in the Analysis file.


In [28]:
# IMPORT STATEMENTS
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from urllib.parse import urljoin
import requests as r
import pandas as pd
import datetime as dt

CURRENT_YEAR = int(dt.datetime.today().strftime("%Y"))
EARLIEST_YEAR = 2003 # checked sources to see how far the data goes back

### Gathering the Win Loss Information


In [1]:
def read_data(year, game_type):
    """
    Reads the data from the website extracting the values of the table based on the year
    Param: 
    year - year of data to be collected
    game_type: either playoff ('playoff') or regular season ("regular_season")

    Return: dataframe of the regular season win loss total for NFL teams
    """
    url = f"https://www.teamrankings.com/nfl/trends/win_trends/?sc=is_{game_type}&range=yearly_"
    url = url + str(year)
    response = r.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    specific_div = soup.find('table') # finding the table with the record of each team
    rows = specific_div.find_all('tr') # get the rows of data

    # Now that we have the data as a list of rows, we can parse the data to construct a data frame
    data = []
    for row in rows:
            cells = row.find_all(['td', 'th'])  # 'td' for regular cells, 'th' for header cells
            row_data = [cell.text.strip() for cell in cells] #extract the contents in each cell
            data.append(row_data)
    columns = data[0]
    df = pd.DataFrame(data[1:], columns=columns)
    df["Year"] = year #helps identify the data points based on year
    return df

In [2]:
def capture_all_data(game_type):
    all_data = []
    CURRENT_YEAR = int(dt.datetime.today().strftime("%Y"))
    for i in range(EARLIEST_YEAR, CURRENT_YEAR):
        x = read_data(i, game_type)
        all_data.append(x) # adding data frame objects to a list
    data = pd.concat(all_data)
    data[["Wins", "Losses", "Ties"]] = data['Win-Loss Record'].apply(lambda x: pd.to_numeric(pd.Series(x.split('-'))))
    return data


Now that I have the functions created I will find the team records for playoffs and regular season

In [5]:
playoff_win_loss = capture_all_data('playoff')
regular_seasonn_win_loss = capture_all_data('regular_season')


## Cleaning Team Records Data

Combining playoff data and regular season data and creating a boolean column for whether teams made it to the playoffs to help with my analysis.

In [107]:

playoff_win_loss = playoff_win_loss[["Team", "Year", "Wins", "Losses"]]
team_records = regular_seasonn_win_loss.merge(playoff_win_loss, how='left', on=["Team", "Year"])

team_records = team_records.fillna(0)
team_records["Playoffs?"] = pd.to_numeric(team_records["Wins_y"]) + pd.to_numeric(team_records["Losses_y"]) > 0


In [111]:
playoff_win_loss[playoff_win_loss['Year'] == 2006].head(10)

,Team,Year,Wins,Losses
0,Green Bay,2006,1,0
1,Indianapolis,2006,5,0
2,Carolina,2006,1,0
3,LA Rams,2006,1,0
4,San Francisco,2006,1,0
5,Pittsburgh,2006,1,0
6,Detroit,2006,1,0
7,Houston,2006,1,0
8,New England,2006,3,1
9,Seattle,2006,2,1


Identifying which teams won the superbowl in the given year

In [113]:
team_records['Superbowl?'] = False
team_records.loc[(team_records['Playoffs?']) & (team_records['Losses_y'] == 0.0), 'Superbowl?'] = True


In [114]:
team_records[team_records['Superbowl?']]

,Team,Win-Loss Record,Win %,MOV,ATS +/-,Year,Wins_x,Losses_x,Ties,Wins_y,Losses_y,Playoffs?,Superbowl?
0,New England,14-2-0,87.5%,6.9,+5.1,2003,14,2,0,3.0,0.0,True,True
33,New England,14-2-0,87.5%,11.1,+4.2,2004,14,2,0,3.0,0.0,True,True
68,Pittsburgh,11-5-0,68.8%,8.2,+4.2,2005,11,5,0,4.0,0.0,True,True
99,Indianapolis,11-4-0,73.3%,4.1,-1.8,2006,11,4,0,5.0,0.0,True,True
111,Green Bay,7-8-0,46.7%,-5.6,-2.6,2006,7,8,0,1.0,0.0,True,True
113,Carolina,7-8-0,46.7%,-3.0,-5.0,2006,7,8,0,1.0,0.0,True,True
114,LA Rams,7-8-0,46.7%,-2.3,-1.4,2006,7,8,0,1.0,0.0,True,True
117,Pittsburgh,7-8-0,46.7%,2.1,-1.3,2006,7,8,0,1.0,0.0,True,True
120,San Francisco,6-9-0,40.0%,-7.8,-2.7,2006,6,9,0,1.0,0.0,True,True
123,Houston,5-10-0,33.3%,-7.1,-0.7,2006,5,10,0,1.0,0.0,True,True


When looking at the above code, there is a clear anomaly in the year 2006 where 7 teams supposedly won the superdowl. However this cannot be true. Many of those teams like 'Detroit' have only 2 wins, yet have a playoff win, which is highly unlikely for a team to even be in the playoffs with that win total.

When looking at the data on the website: https://www.teamrankings.com/nfl/trends/win_trends/?sc=is_playoff&range=yearly_2006 we can see that there is clearly an error with the data being shown on this page. Due to this anomaly I will exclude 2006 win loss data and team statistics data in my analysis.

In [23]:
new_columns = {}
for column in team_records.columns:
    if column.endswith('_x'):
        new_columns[column] = f"Reg_Szn_{column.replace('_x', '')}"
    elif column.endswith('_y'):
        new_columns[column] = f"Playoff_{column.replace('_y', '')}"
team_records.rename(columns = new_columns, inplace=True)


In [24]:
team_records.columns

Index(['Team', 'Win-Loss Record', 'Win %', 'MOV', 'ATS +/-', 'Year',
       'Reg_Szn_Wins', 'Reg_Szn_Losses', 'Ties', 'Playoff_Wins',
       'Playoff_Losses', 'Playoffs?', 'Superbowl?'],
      dtype='object')

Saves the team records into 1 .csv file

In [25]:
team_records.to_csv("C:/Users/rchap/Git/NFL_TEAM_DATA/NFL_Team_Records.csv", index=False)

### Saving information about team statistics into a .csv file
- creating function to scrape team stats
- combining information about offense and defense into a singular table
- save two files as a single .csv

In [29]:
def collect_team_stats(key):
    """
    Function that calculates team statistics based on input 'key' which is a
    dictionary of columns from the nfl.com/stats/team-stats website.
    """
    if len(key) > 2:
        side = 'defense'
    else:
        side = 'offense'
    for i in key.keys():
        for year in range(EARLIEST_YEAR, CURRENT_YEAR):
            url = f"https://www.nfl.com/stats/team-stats/{side}"
            url = url + (f'/{i}/{year}/reg/all')
            response = r.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            specific_div = soup.find('table')
            rows = specific_div.find_all('tr')
            data = []
            for row in rows:
                cells = row.find_all(['td', 'th'])  # 'td' for regular cells, 'th' for header cells
                row_data = [cell.text.strip() for cell in cells] #extract the contents in each cell
                data.append(row_data)
            columns = data[0]
            data_table = pd.DataFrame(data[1:], columns=columns)
            data_table["Year"] = year
            tables = key[i]
            tables.append(data_table)
        df = pd.concat(tables)
        df['Team'] = df['Team'].apply(lambda x: x.split('\n')[0])
        if 'Lng' in df.columns:
            df['Lng'] = df['Lng'].apply(lambda x: str(x).split('T')[0])
        key[i] = df
    return key

In [39]:
defense_data_dict = {'Passing': [], 'Rushing': [], 'Downs': [], 'Fumbles': [], 'Interceptions': []}
collect_team_stats(defense_data_dict)


{'Passing':           Team  Att  Cmp Cmp % Yds/Att   Yds  TD INT  Rate  1st  1st% 20+ 40+  \
 0       Giants  519  309  59.5     6.6  3412  25  10  87.1  178  34.3  59   3   
 1         Rams  510  296    58       6  3072  23  24    71  164  32.2  43   6   
 2   Buccaneers  475  274  57.7     5.7  2710  16  20  67.6  140  29.5  30   6   
 3        Lions  522  331  63.4     6.8  3578  26  15  88.1  166  31.8  42   7   
 4        Colts  445  277  62.2     6.3  2809  18  15  79.7  138    31  31   8   
 ..         ...  ...  ...   ...     ...   ...  ..  ..   ...  ...   ...  ..  ..   
 27     Broncos  572  382  66.8     7.5  4302  29  11    98  201  35.1  56   8   
 28    Steelers  573  348  60.7     7.3  4164  23  16  84.7  186  32.5  63   6   
 29      Ravens  634  384  60.6     5.9  3717  18  18  74.6  175  27.6  46   6   
 30      Titans  571  387  67.8     7.3  4181  20   6  96.4  189  33.1  52  11   
 31       49ers  621  410    66     6.4  3949  20  22  79.6  190  30.6  45   5   
 
   

In [31]:
defense_data_dict["Rushing"]

,Team,Att,Rush Yds,YPC,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM,Year
0,Ravens,448,1536,3.4,6,3,0,36,78,17.4,6,2003
1,Broncos,379,1605,4.2,11,11,3,98,77,20.3,14,2003
2,Panthers,434,1722,4,10,8,2,43,83,19.1,8,2003
3,Cowboys,413,1425,3.4,7,5,1,64,68,16.5,7,2003
4,Packers,413,1701,4.1,10,7,3,67,91,22,9,2003
...,...,...,...,...,...,...,...,...,...,...,...,...
27,Vikings,446,1678,3.8,14,2,1,43,104,23.3,7,2023
28,Jets,517,2108,4.1,14,10,2,48,111,21.5,13,2023
29,Bengals,456,2145,4.7,17,17,1,41,127,27.8,6,2023
30,Seahawks,515,2352,4.6,24,18,4,72,143,27.8,3,2023


In [85]:
defense_stats = defense_data_dict["Passing"]
for key in defense_data_dict.keys():
    if key != "Passing":
        defense_stats = defense_stats.merge(defense_data_dict[key],
                                            how='left', on=["Team", "Year"])

In [86]:
defense_stats

,Team,Att_x,Cmp,Cmp %,Yds/Att,Yds,TD_x,INT_x,Rate,1st,...,Scrm Plys,FF,FR,FR TD,Rec FUM,Rush FUM_y,INT_y,INT TD,INT Yds,Lng
0,Giants,519,309,59.5,6.6,3412,25,10,87.1,178,...,1060,21,0,0,4,7,10,2,199,56
1,Rams,510,296,58,6,3072,23,24,71,164,...,964,25,0,0,5,9,24,3,505,74
2,Buccaneers,475,274,57.7,5.7,2710,16,20,67.6,140,...,962,23,0,0,5,10,20,3,229,44
3,Lions,522,331,63.4,6.8,3578,26,15,88.1,166,...,997,17,0,0,5,12,15,1,130,48
4,Colts,445,277,62.2,6.3,2809,18,15,79.7,138,...,913,24,0,0,3,10,15,2,244,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,Broncos,572,382,66.8,7.5,4302,29,11,98,201,...,1083,19,15,1,2,12,11,0,85,33
668,Steelers,573,348,60.7,7.3,4164,23,16,84.7,186,...,1074,19,11,1,6,8,16,1,184,32
669,Ravens,634,384,60.6,5.9,3717,18,18,74.6,175,...,1109,16,13,0,3,4,18,1,258,52
670,Titans,571,387,67.8,7.3,4181,20,6,96.4,189,...,1096,13,8,0,2,3,6,1,86,44


In [87]:
new_columns = {}
for column in defense_stats.columns:
    if column != 'Team' and column != 'Year':
        if column.endswith('_x'):
            new_columns[column] = f"Def_Passing_{column.replace('_x', '')}"
        elif column.endswith('_y'):
            new_columns[column] = f"Def_{column.replace('_y', '')}"
        else:
            new_columns[column] = f"Def_{column}"
defense_stats.rename(columns = new_columns, inplace=True)

In [88]:
defense_stats.columns

Index(['Team', 'Def_Passing_Att', 'Def_Cmp', 'Def_Cmp %', 'Def_Yds/Att',
       'Def_Yds', 'Def_Passing_TD', 'Def_Passing_INT', 'Def_Rate', 'Def_1st',
       'Def_1st%', 'Def_Passing_20+', 'Def_Passing_40+', 'Def_Passing_Lng',
       'Def_Sck', 'Year', 'Def_Att', 'Def_Rush Yds', 'Def_YPC', 'Def_TD',
       'Def_20+', 'Def_40+', 'Def_Lng', 'Def_Passing_Rush 1st',
       'Def_Passing_Rush 1st%', 'Def_Passing_Rush FUM', 'Def_3rd Att',
       'Def_3rd Md', 'Def_4th Att', 'Def_4th Md', 'Def_Rec 1st',
       'Def_Rec 1st%', 'Def_Rush 1st', 'Def_Rush 1st%', 'Def_Scrm Plys',
       'Def_FF', 'Def_FR', 'Def_FR TD', 'Def_Rec FUM', 'Def_Rush FUM',
       'Def_INT', 'Def_INT TD', 'Def_INT Yds', 'Def_Lng'],
      dtype='object')

In [81]:
offense_data_dict = {'Passing': [], 'Rushing': []}
offense_data_dict = collect_team_stats(offense_data_dict)

In [82]:
off_passing_data = offense_data_dict['Passing']
off_rushing_data = offense_data_dict['Rushing']
off_passing_data

,Team,Att,Cmp,Cmp %,Yds/Att,Pass Yds,TD,INT,Rate,1st,1st%,20+,40+,Lng,Sck,SckY,Year
0,Giants,616,344,55.8,5.9,3642,16,20,68.4,184,29.9,35,8,77,44,259,2003
1,Rams,600,377,62.8,7.2,4287,23,23,81,211,35.2,64,6,48,43,326,2003
2,Buccaneers,592,369,62.3,6.7,3941,27,22,81.5,190,32.1,38,9,76,23,136,2003
3,Lions,588,319,54.2,5.1,2988,17,24,61.1,152,25.8,26,3,72,11,64,2003
4,Colts,569,381,67,7.5,4289,29,10,99,212,37.3,45,9,79,19,110,2003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,Broncos,513,337,65.7,7,3566,28,9,96.7,154,30,44,13,60,52,304,2023
28,Steelers,506,323,63.8,6.8,3421,13,9,84.6,153,30.2,41,9,86,36,258,2023
29,Ravens,494,328,66.4,7.9,3881,27,7,102.5,180,36.4,52,9,80,41,246,2023
30,Titans,494,304,61.5,7.1,3512,14,11,83.2,158,32,54,10,70,64,445,2023


In [83]:
offensive_stats = pd.merge(off_passing_data, off_rushing_data, how='left',
                           on=["Team", "Year"])
new_columns = {}
for column in offensive_stats.columns:
    if column != 'Year' and column != 'Team':
        if column.endswith('_x'):
            new_columns[column] = f"Off_Pass_{column.replace('_x', '')}"
        elif column.endswith('_y'):
            new_columns[column] = f"Off_Rush_{column.replace('_y', '')}"
        else:
            new_columns[column] = f"Off_{column}"
offensive_stats.rename(columns = new_columns, inplace=True)
offensive_stats

,Team,Off_Pass_Att,Off_Cmp,Off_Cmp %,Off_Yds/Att,Off_Pass Yds,Off_Pass_TD,Off_INT,Off_Rate,Off_1st,...,Off_Rush_Att,Off_Rush Yds,Off_YPC,Off_Rush_TD,Off_Rush_20+,Off_Rush_40+,Off_Rush_Lng,Off_Rush 1st,Off_Rush 1st%,Off_Rush FUM
0,Giants,616,344,55.8,5.9,3642,16,20,68.4,184,...,387,1559,4,6,6,0,27,89,23,9
1,Rams,600,377,62.8,7.2,4287,23,23,81,211,...,411,1496,3.6,19,7,1,52,97,23.6,5
2,Buccaneers,592,369,62.3,6.7,3941,27,22,81.5,190,...,421,1648,3.9,5,3,2,61,86,20.4,6
3,Lions,588,319,54.2,5.1,2988,17,24,61.1,152,...,376,1338,3.6,5,8,0,39,69,18.4,7
4,Colts,569,381,67,7.5,4289,29,10,99,212,...,453,1695,3.7,16,5,1,43,104,23,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,Broncos,513,337,65.7,7,3566,28,9,96.7,154,...,451,1810,4,8,9,0,38,116,25.7,5
668,Steelers,506,323,63.8,6.8,3421,13,9,84.6,153,...,487,2010,4.1,16,14,1,74,115,23.6,9
669,Ravens,494,328,66.4,7.9,3881,27,7,102.5,180,...,541,2661,4.9,26,21,4,60,144,26.6,7
670,Titans,494,304,61.5,7.1,3512,14,11,83.2,158,...,444,1846,4.2,16,10,2,69,103,23.2,6


Now that we have the offensive and defensive statistics, I will combine them into one file.

In [92]:
team_stats = pd.merge(offensive_stats, defense_stats, how='left', on=["Team", "Year"])

To connect the "NFL_Team_Records"" and "NFL_Team_Stats" data we need another connecting table. This is because the team records data has **the city name for each 'team'**. The team stats data has the **'team' column show up as mascots**. To match these columns I have created a manual column that connects city name to the team name mascots. 
I manually made this because Teams like the Washington Commanders have had different names since 2003 and there are not that many teams (only 32). Creating an associative entity is the best way to accomplish this.

In [93]:
city_to_team = pd.read_csv("City_Mascot_Connection.csv")
city_to_team

,City,Team
0,Arizona,Cardinals
1,Atlanta,Falcons
2,Baltimore,Ravens
3,Buffalo,Bills
4,Carolina,Panthers
5,Chicago,Bears
6,Cincinnati,Bengals
7,Cleveland,Browns
8,Dallas,Cowboys
9,Denver,Broncos


When looking at the 'NFL_Team_Records' data, the city names have been updated to the current city names for teams that have moved. For examples, data about the "Las Vegas Raiders" which was the "Oakland Raiders" until recently is recorded with "Las Vegas Raiders" for all the years since 2003. Thus I want to use the __'Team'__ column in the __'NFL_Team_Records'__ table as my field for team in all of the analysis I do.

In [94]:
team_records = pd.read_csv("NFL_Team_Records.csv")
team_records['Team'].unique()

array(['New England', 'Kansas City', 'Tennessee', 'Indianapolis',
       'LA Rams', 'Philadelphia', 'Carolina', 'Seattle', 'Denver',
       'Miami', 'Dallas', 'Green Bay', 'Baltimore', 'Minnesota',
       'New Orleans', 'Cincinnati', 'Chicago', 'Tampa Bay',
       'San Francisco', 'NY Jets', 'Buffalo', 'Pittsburgh', 'Cleveland',
       'Detroit', 'Jacksonville', 'Houston', 'Atlanta', 'Washington',
       'Las Vegas', 'LA Chargers', 'NY Giants', 'Arizona'], dtype=object)

I will use the 'city_mascot_connection' table to add the 'City' to the "NFL_Team_Stats" table so that we can merge the data in the future if needed. 

In [ ]:
team_stats = pd.read_csv("NFL_Team_Stats.csv")
team_stats = pd.merge(team_stats, city_to_team, how='left', left_on='Team', right_on='Team')


Now we have a 'City' column in the team_stats data that will allow me to analyze team stats and team records.

In [98]:
team_stats.to_csv("NFL_Team_Stats.csv", index=False)